In [110]:
import torch
import ssl
import pandas as pd

ssl._create_default_https_context = ssl._create_unverified_context
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.io import read_image

import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision.io import read_image
from torchvision.transforms import v2
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision import models as torchvision_models
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import resnet101, ResNet101_Weights
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from torchvision.models import convnext_base, ConvNeXt_Base_Weights
from torchvision.models import vit_b_16, ViT_B_16_Weights

import torch
import torchvision.transforms as T
import json
import urllib
import requests
from PIL import Image
from io import BytesIO


In [111]:
# Set a device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Load models

In [112]:
accessoire_dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14_reg')
inner_wear_dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14_reg')
outer_wear_dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14_reg')
bottom_wear_dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14_reg')
shoes_dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14_reg')

Using cache found in /Users/romanzberg/.cache/torch/hub/facebookresearch_dinov2_main
Using cache found in /Users/romanzberg/.cache/torch/hub/facebookresearch_dinov2_main
Using cache found in /Users/romanzberg/.cache/torch/hub/facebookresearch_dinov2_main
Using cache found in /Users/romanzberg/.cache/torch/hub/facebookresearch_dinov2_main
Using cache found in /Users/romanzberg/.cache/torch/hub/facebookresearch_dinov2_main


In [113]:
accessoire_dinov2_vitb14.embed_dim

384

# freeze weights of models

In [114]:
for param in accessoire_dinov2_vitb14.parameters():
    param.requires_grad = False
for param in inner_wear_dinov2_vitb14.parameters():
    param.requires_grad = False
for param in outer_wear_dinov2_vitb14.parameters():
    param.requires_grad = False
for param in bottom_wear_dinov2_vitb14.parameters():
    param.requires_grad = False
for param in shoes_dinov2_vitb14.parameters():
    param.requires_grad = False

# Testing model

In [115]:
# Get ImageNet labels
imagenet_class_url = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
imagenet_classes = json.loads(urllib.request.urlopen(imagenet_class_url).read())


# # Set a device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#https://www.modelbit.com/blog/deploying-dinov2-for-image-classification-with-modelbit#:~:text=To%20do%20this%2C%20simply%20use,machine%20to%20your%20Colab%20directory.&text=Next%2C%20you'll%20want%20to,ImageNet%20preprocessing%20on%20the%20image.&text=Now%2C%20we%20can%20pass%20the,a%20class%20ID%20and%20label.
def dinov2_classifier(img_url):
    response = requests.get(img_url)
    image = Image.open(BytesIO(response.content))

    # Preprocess the image
    transform = T.Compose([
        T.Resize(256, interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])
    image = transform(image)

    # Move the image to the GPU if available
    image = image.to(device)

    # Extract the features
    with torch.no_grad():
        features = accessoire_dinov2_vitb14(image.unsqueeze(0))

    # Print the features
    return {'index': features.argmax(-1).item(),
            'label': imagenet_classes[features.argmax(-1).item()]
            }

In [116]:
dinov2_classifier(
    "https://www.baunat.com/de/15260067_CG-GY-PG-0050R_1_With-chain_722x722/2-00-karat-solitar-anhanger-aus-gelbgold-mit-rundem-diamant-und-4-krappen.jpg")

{'index': 229, 'label': 'Old English Sheepdog'}

# Implementing Custom Model

In [117]:
model_config = {
    'use_n_blocks': 1,
    'init_lr': 0.0002,
    'epochs': 20,
}


# class for DINOv2
class ModelWithIntermediateLayers(nn.Module):
    def __init__(self, feature_model, n_last_blocks):
        super().__init__()
        self.feature_model = feature_model
        self.feature_model.eval()
        self.n_last_blocks = n_last_blocks

    def forward(self, image):
        with torch.inference_mode():
            features = self.feature_model.get_intermediate_layers(
                image, self.n_last_blocks, return_class_token=True
            )
        return features


def create_linear_input(x_tokens_list, use_n_blocks, use_avgpool):
    intermediate_output = x_tokens_list[-use_n_blocks:]
    output = torch.cat([class_token for _, class_token in intermediate_output], dim=-1)
    if use_avgpool:
        output = torch.cat(
            (
                output,
                torch.mean(intermediate_output[-1][0], dim=1),  # patch tokens
            ),
            dim=-1,
        )
        output = output.reshape(output.shape[0], -1)
    return output.float()


class OutfitClassifier(nn.Module):

    @property
    def embed_dim(self):
        return self._embed_dim

    def __init__(self):
        super(OutfitClassifier, self).__init__()
        self._accessoire_model = ModelWithIntermediateLayers(accessoire_dinov2_vitb14, model_config['use_n_blocks'])
        self._inner_wear_model = ModelWithIntermediateLayers(inner_wear_dinov2_vitb14, model_config['use_n_blocks'])
        self._outer_wear_model = ModelWithIntermediateLayers(outer_wear_dinov2_vitb14, model_config['use_n_blocks'])
        self._bottom_wear_model = ModelWithIntermediateLayers(bottom_wear_dinov2_vitb14, model_config['use_n_blocks'])
        self._shoes_model = ModelWithIntermediateLayers(shoes_dinov2_vitb14, model_config['use_n_blocks'])
        self._embed_dim = accessoire_dinov2_vitb14.embed_dim * (model_config['use_n_blocks']) * 5

        self._accessoire_model.to(device)
        self._inner_wear_model.to(device)
        self._outer_wear_model.to(device)
        self._bottom_wear_model.to(device)
        self._shoes_model.to(device)

        self.trainable_model = nn.Sequential(
            nn.Linear(self._embed_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

        self.trainable_model.eval()
        self.trainable_model.to(device)
        self.counter = 0

    def forward(self, X):
        number_of_features = len(X)
        # print()
        # print(X[0].shape)
        # print(X[1].shape)
        # print(X[2].shape)
        # print(X[3].shape)
        # print(X[4].shape)
        # print(X)
        feature_list = []
        # for x in X:
        # x = X[0]

        number_of_rows = X[0].shape[0]
        # print(f'number of rows: {number_of_rows}')
        with torch.no_grad():
            for index in range(number_of_rows):
                extracted_features = torch.cat(
                    (
                        create_linear_input(self._accessoire_model.forward(X[0][index]), model_config['use_n_blocks'],
                                            False),
                        create_linear_input(self._inner_wear_model.forward(X[1][index]), model_config['use_n_blocks'],
                                            False),
                        create_linear_input(self._outer_wear_model.forward(X[2][index]), model_config['use_n_blocks'],
                                            False),
                        create_linear_input(self._bottom_wear_model.forward(X[3][index]), model_config['use_n_blocks'],
                                            False),
                        create_linear_input(self._shoes_model.forward(X[4][index]), model_config['use_n_blocks'], False)
                    ),
                    dim=1
                )
                extracted_features.requires_grad_()

                feature_list.append(extracted_features)

        predictions = []
        # print(f'length of featurs: {len(feature_list)}')
        for features in feature_list:
            prediction = self.trainable_model.forward(features)
            predictions.append(torch.squeeze(prediction).nan_to_num(0))

        self.counter += 1
        print(f'finished {self.counter * 8}')

        return torch.squeeze(torch.tensor(predictions)).to(torch.float)

        # return self.trainable_model.forward(extracted_features)


class Classifier(nn.Module):
    """Linear layer to train on top of frozen features"""

    def __init__(self, out_dim, config, num_classes=2):
        super().__init__()
        self.out_dim = out_dim
        self.use_n_blocks = config['use_n_blocks']
        self.num_classes = num_classes

    def forward(self, input):
        output = create_linear_input(input, self.use_n_blocks, False)
        return self.model(output)


outfit_classifier = OutfitClassifier()
classifier = Classifier(outfit_classifier.embed_dim, model_config)
classifier = classifier.to(device)

In [118]:
# Class for test dataset
def get_image(img_path, tfms: v2.Compose):
    if img_path is None:
        return torch.empty(3, 224, 224)
    else:
        return tfms(read_image(f'../datasets/{img_path}'))


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data_frame, tfms: v2.Compose):
        self.df = data_frame
        self.tfms = tfms

    def __getitem__(self, index):
        outfit = self.df.iloc[index]
        img_accessoire = get_image(outfit['Accessoire_imagePath'], self.tfms)
        img_innerwear = get_image(outfit['Innerwear_imagePath'], self.tfms)
        img_bottomwear = get_image(outfit['Bottomwear_imagePath'], self.tfms)
        img_shoe = get_image(outfit['Shoes_imagePath'], self.tfms)
        img_outerwear = get_image(outfit['Outerwear_imagePath'], self.tfms)

        # print(f'accessoire shape: {img_accessoire.shape}; path={outfit["Accessoire_imagePath"]}')
        # print(f'innerwear shape: {img_innerwear.shape}; path={outfit["Innerwear_imagePath"]}')

        target_variable = torch.tensor(outfit['valid_outfit']).to(torch.float)
        target_variable.requires_grad_()

        return (
                   img_accessoire.unsqueeze(0),
                   img_innerwear.unsqueeze(0),
                   img_bottomwear.unsqueeze(0),
                   img_shoe.unsqueeze(0),
                   img_outerwear.unsqueeze(0)
               ), target_variable

    def __len__(self):
        return df.shape[0]

In [119]:
df = pd.read_parquet(
    '../datasets/imageBasedModel/polyvore/polyvore_63eb50dc58d97415384467bef7b3c9e1bd6c96e06ad19571b6bc15e9dd5af262.parquet'
)

df

,Innerwear_imagePath,Bottomwear_imagePath,Accessoire_imagePath,Shoes_imagePath,Outerwear_imagePath,valid_outfit
120161271,raw/images/120161271/1.jpg,raw/images/120161271/2.jpg,None,None,None,0
143656996,raw/images/143656996/1.jpg,raw/images/143656996/3.jpg,raw/images/143656996/5.jpg,None,None,0
216470135,raw/images/216470135/1.jpg,raw/images/216470135/2.jpg,None,raw/images/216470135/3.jpg,None,1
216220312,raw/images/216220312/1.jpg,raw/images/216220312/2.jpg,raw/images/216220312/4.jpg,raw/images/216220312/3.jpg,None,1
192203629,raw/images/192203629/2.jpg,raw/images/192203629/3.jpg,raw/images/192203629/5.jpg,None,raw/images/192203629/1.jpg,0
...,...,...,...,...,...,...
201717504,raw/images/201717504/1.jpg,raw/images/201717504/2.jpg,raw/images/201717504/4.jpg,raw/images/201717504/3.jpg,None,1
216589548,raw/images/216589548/1.jpg,raw/images/216589548/3.jpg,raw/images/216589548/5.jpg,None,raw/images/216589548/2.jpg,1
216860218,raw/images/216860218/1.jpg,raw/images/216860218/3.jpg,raw/images/216860218/5.jpg,None,raw/images/216860218/2.jpg,1
211099953,raw/images/211099953/1.jpg,raw/images/211099953/2.jpg,None,raw/images/211099953/3.jpg,None,1


In [134]:
CROP_SIZE = 256
BATCH_SIZE = 8
NUM_WORKERS = 0

data_transforms = v2.Compose([
    v2.PILToTensor(),
    T.Resize(256, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(224),
    v2.AutoAugment(v2.AutoAugmentPolicy.IMAGENET),
    v2.ConvertImageDtype(torch.float32),
    v2.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

generator1 = torch.Generator().manual_seed(42)

image_datasets = {}
image_datasets['train'], image_datasets['val'], image_datasets['test'] = torch.utils.data.random_split(
    CustomDataset(df, data_transforms), [0.8, 0.10, 0.10], generator=generator1)

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE,
                                              shuffle=True, num_workers=NUM_WORKERS)
               for x in ['train', 'val']}


In [135]:
def train_loop(dataloader, feature_model, loss_fn, optimizer):
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    feature_model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    running_loss = 0.0
    running_corrects = 0
    for batch, (X, y) in enumerate(dataloader):
        X = [x.to(device) for x in X]
        y = y.to(device)
        # Compute prediction and loss
        pred = feature_model(X)
        # print('pred:', pred)
        # print('y:', y)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if torch.cuda.is_available():
            torch.cuda.synchronize()

        # Statistics
        running_loss += loss.item()
        running_corrects += (pred.round() == y).type(torch.float).sum().item()

    epoch_loss = running_loss / num_batches
    epoch_acc = running_corrects / size
    return epoch_acc, epoch_loss

In [136]:
@torch.inference_mode()
def val_loop(dataloader, feature_model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    feature_model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    val_loss, val_acc = 0.0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X
            y = y
            pred = feature_model(X)
            val_loss += loss_fn(pred, y).item()
            val_acc += (pred.round() == y).type(torch.float).sum().item()

    val_loss /= num_batches
    val_acc /= size
    return val_acc, val_loss

In [137]:
outfit_classifier = OutfitClassifier()

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(
    outfit_classifier.parameters(),
    lr=model_config['init_lr'],
    betas=(0.9, 0.999)
)
# optimizer = torch.optim.SGD(
#     linear_classifier.parameters(),
#     lr=model_config['init_lr'],
#     momentum=0.9,
#     weight_decay=0, # we do not apply weight decay
# )

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, model_config['epochs'], eta_min=0)
pytorch_total_params = sum(p.numel() for p in outfit_classifier.parameters() if p.requires_grad)
print(f'total trainable params: {pytorch_total_params}')

total trainable params: 30753


In [138]:
import numpy as np

VAL_FREQ = 1
best_acc = 0.0
best_acc_loss = np.inf
train_data = []
for t in range(model_config['epochs']):
    print(f'Epoch {t + 1}\n-------------------------------')

    train_acc, train_loss = train_loop(dataloaders['train'], outfit_classifier, loss_fn, optimizer)
    train_data.append({
        'phase': 'train',
        'epoch': t,
        'lr': optimizer.param_groups[0]["lr"],
        'accuracy': train_acc,
        'loss': train_loss
    })
    scheduler.step()
    print(f'Train:\n  train_acc = {train_acc}, train_loss = {train_loss}')
    if t % VAL_FREQ == 0 or t == model_config['epochs'] - 1:
        val_acc, val_loss = val_loop(dataloaders['val'], outfit_classifier, loss_fn)
        train_data.append({
            'phase': 'val',
            'epoch': t,
            'lr': optimizer.param_groups[0]["lr"],
            'accuracy': val_acc,
            'loss': val_loss
        })
        print(f'Validation:\n    val_acc = {val_acc}, val_loss = {val_loss}')
        if (val_acc == best_acc and val_loss < best_acc_loss) or (val_acc > best_acc):
            best_acc, best_acc_loss = val_acc, val_loss
            print(f'Best accuracy = {best_acc}, best_loss = {best_acc_loss}')
            save_dict = {
                'epoch': t + 1,
                'state_dict': outfit_classifier.state_dict(),
                'optimizer': optimizer.state_dict(),
                # 'scheduler': scheduler.state_dict(),
                'best_acc': best_acc,
                'best_loss': best_acc_loss
            }
            torch.save(save_dict, os.path.join('.', 'dino_classifier_ckpt.pth'))

    print('\n')
print('Training completed.')

Epoch 1
-------------------------------
finished 8
finished 16
finished 24
finished 32
finished 40
finished 48
finished 56
finished 64
finished 72
finished 80
finished 88
finished 96
finished 104
finished 112
finished 120
finished 128
finished 136
finished 144
finished 152
finished 160
finished 168
finished 176
finished 184
finished 192
finished 200
finished 208
finished 216
finished 224
finished 232
finished 240
finished 248
finished 256
finished 264
finished 272
finished 280
finished 288
finished 296
finished 304
finished 312
finished 320
finished 328
finished 336
finished 344
finished 352
finished 360
finished 368
finished 376
finished 384
finished 392
finished 400
finished 408
finished 416
finished 424
finished 432
finished 440
finished 448
finished 456
finished 464
finished 472
finished 480
finished 488
finished 496
finished 504
finished 512
finished 520
finished 528
finished 536
finished 544
finished 552
finished 560
finished 568
finished 576
finished 584
finished 592
finished 60

KeyboardInterrupt: 